In [15]:
import os
from PIL import Image, ImageDraw

class IAMDataset():
    def __init__(self, dataset_dir):
        self.root_dir = dataset_dir
        self.image_paths = []
        self.annotations = {}
        self._load_data()

    def _load_data(self):
        # Iterate through files in the dataset directory
        image_path = os.path.join(self.root_dir, 'formsA-D')
        for file_name in os.listdir(image_path):
            if file_name.endswith('.png'):
                self.image_paths.append(file_name)
                self.annotations[file_name] = []

        # Load annotations from words.txt
        annotation_path = os.path.join(self.root_dir, 'ascii/words.txt')
        with open(annotation_path, 'r') as f:
            lines = f.readlines()

        # Process annotations
        for line in lines:
            if not line.startswith('#'):
                parts = line.strip().split()
                image_id = parts[0].split("-")[0] + "-" + parts[0].split("-")[1]
                image_path = f'{image_id}.png'
                if image_path in self.annotations:
                    self.annotations[image_path].append(parts[3:7])  # Append bounding box coordinates

    def display_image(self, image_id):
        image_id = self.image_paths[image_id]
        image_path = os.path.join(self.root_dir, 'formsA-D')
        image_path = os.path.join(image_path, image_id)

        # Load image
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # Find highest and lowest bounding boxes
        highest_y_min = float('inf')
        lowest_y_max = -1
        for bbox in self.annotations[image_id]:
            y_min = int(bbox[1])
            y_max = y_min + int(bbox[3])
            highest_y_min = min(highest_y_min, y_min)
            lowest_y_max = max(lowest_y_max, y_max)
        
        threshold=10
        # Add threshold
        highest_y_min = max(0, highest_y_min - threshold)
        lowest_y_max = min(image.height, lowest_y_max + threshold)

        # Crop image
        image = image.crop((0, highest_y_min, image.width, lowest_y_max))

        # Draw bounding boxes
        draw = ImageDraw.Draw(image)
        for bbox in self.annotations[image_id]:
            x_min, y_min, width, height = map(int, bbox)
            x_max = x_min + width
            y_max = y_min + height
            assert x_min < x_max
            draw.rectangle([x_min, y_min - highest_y_min, x_max, y_max - highest_y_min], outline="red")

        # Show image
        image.show()

In [16]:
# Path to the folder containing images
images_folder = r'C:\Users\IsaacStalley\Documents\GitHub\IAM-dataset'
annotation_path = r'C:\Users\IsaacStalley\Documents\GitHub\IAM-dataset\ascii\words.txt'

dataset = IAMDataset(images_folder)
dataset.display_image(1)  # Display the first image with annotations

AssertionError: 